# RAG Pipeline

This notebook is our pipeline to transform the various websites around CS to text and put them in the vector store.

In [1]:
import chromadb
from chromadb.utils.embedding_functions import Bm25EmbeddingFunction, DefaultEmbeddingFunction, HuggingFaceEmbeddingFunction
from chromadb import Documents, EmbeddingFunction, Embeddings
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os 

load_dotenv()

True

In [2]:
URL = "https://api.infomaniak.com/1/ai/models"
headers = {
  'Authorization': f'Bearer {os.getenv("API_TOKEN")}',
  'Content-Type': 'application/json',
}
req = requests.request("GET", url = URL , headers = headers)
res = req.json()

info_df = pd.DataFrame(res["data"])
info_df[info_df["type"] == "embedding"]

,id,name,type,documentation_link,description,info_status,logo_url,last_updated_at,max_token_input,version,meta
6,12,bge_multilingual_gemma2,embedding,https://developer.infomaniak.com/docs/api/post...,Bge Multilingual Gemma2,coming_soon,https://storage4.infomaniak.com/ai-tools/publi...,2024-12-04,8000.0,1.0,{'is_beta': False}
7,13,mini_lm_l12_v2,embedding,https://developer.infomaniak.com/docs/api/post...,All MiniLM L12 v2,coming_soon,https://storage4.infomaniak.com/ai-tools/publi...,2024-12-04,128.0,2.0,{'is_beta': False}


## 1. Data Processing: Conver HTML to Text

using beautifulSoup and simple get request, we collect the data from websites.

In [3]:
# needed to pretend I am a normal user ;)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
}

def url_to_string(url: str) -> str:
    if not url.startswith("https://"):
        url = "https://" + url
    
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")
    site_text = " ".join([text for text in soup.stripped_strings])
    return site_text


demo_url = "www.orientation.ch/dyn/show/1900?id=152"
site_text = url_to_string(demo_url)

print("Number of chars:", len(site_text))
print("Number of words:", len(site_text.split()))

Number of chars: 30161
Number of words: 3883


In [4]:
df = pd.read_csv("../data/links.csv")

df = df.drop(columns="verified")

url_list = df["url"].to_list()
url_text_list = [url_to_string(url) for url in url_list]

assert len(df["url"]) == len(url_text_list), "not the same number of sites and web pages scraped!"

In [5]:
df["url_text"] = url_text_list
df.head()

,title,url,type,url_text
0,informaticien cfc suisse,www.orientation.ch/dyn/show/1900?id=152,apprentissage,Informaticien CFC / Informaticienne CFC - orie...
1,école techniques des métiers de Lausanne,https://www.etml.ch/formation/em/informaticien...,école professionelle,Apprentissage d'informaticien·ne - ETML Nous u...
2,école des métiers Fribourg,https://emf.ch/formation/informaticien-ne-cfc,école professionelle,Informaticien-ne CFC · Ecole des Métiers Fribo...
3,centre professionel du Nord Vaudois,https://www.cpnv.ch/formations/ecole-metiers/i...,école professionelle,Informaticien-ne CFC - CPNV Aller au contenu M...
4,université de fribourg,www.unifr.ch/inf/fr/informatique,université,Informatique | Département d'informatique | U...


## 2. RAG Benchmark

First two models are on infomaniak.
The rest we will need to train ourselves by getting the model with a GPU.

|evaluate? | Model name | Link |
|:--------:|------------|------|
| |MiniLM L12 v2 | https://developer.infomaniak.com/docs/api/post/1/ai/%7Bproduct_id%7D/openai/v1/embeddings |
| X |BGE Multilingual Gemma 2 | https://developer.infomaniak.com/docs/api/post/1/ai/%7Bproduct_id%7D/openai/v1/embeddings |
| X | paraphrase-multilingual-MiniLM-L12-v2 | https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 |
| X | paraphrase-multilingual-mpnet-base-v2 | https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2 |
| X | distiluse-base-multilingual-cased-v2 | https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2 |
|  | Alibaba-NLP/gte-Qwen2-1.5B-instruct | https://huggingface.co/Alibaba-NLP/gte-Qwen2-1.5B-instruct |
| X | BM25 | chromadb through `fastembed` | 

In [6]:
DB_PATH = "../data/"

client = chromadb.PersistentClient(path=DB_PATH)
# ensure the DB is available
print("hearbeat:", client.heartbeat())

ids: list[str] = df["title"].to_list()
documents: list[str] = df["url_text"].to_list()
metadatas: list[dict[str, str]] = df.loc[:, ["title", "type", "url"]].to_dict("records")

hearbeat: 1760216294561660211


## Infomaniak Model: MultiLingual Gemma 2 and MiniLM-L12-v2

The latter is english only, we should not evaluate it.

In [7]:
PRODUCT_ID = os.getenv("PRODUCT_ID")
API_TOKEN = os.getenv("API_TOKEN")

URL = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1/embeddings"

headers = {
  'Authorization': f"Bearer {API_TOKEN}",
  'Content-Type': 'application/json',
}

payload = {
    "input": documents,
    "model": "bge_multilingual_gemma2",
    "mode": "index",
}

req = requests.post(url=URL , json=payload, headers=headers)
res = req.json()
print(req.status_code)

assert len(documents) == len(res["data"])

200


In [8]:
PRODUCT_ID = os.getenv("PRODUCT_ID")
API_TOKEN = os.getenv("API_TOKEN")


class MultinligualGemma2(EmbeddingFunction):
    def __init__(self) -> None:
        self.model_name = "bge_multilingual_gemma2"
        self.url = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1/embeddings"
        self.headers = {
          'Authorization': f"Bearer {API_TOKEN}",
          'Content-Type': 'application/json',
        }
        
    def __call__(self, input_data: Documents) -> Embeddings:
        payload = {
            "input": input_data,
            "model": self.model_name,
        }

        req = requests.post(url=self.url, json=payload, headers=self.headers)
        res = req.json()
        data = res["data"]
        embeddings = [np.array(x["embedding"]) for x in data]
        
        return embeddings


# Do not evaluate this one
class MiniLML12V2(EmbeddingFunction):
    def __init__(self) -> None:
        self.model_name = "mini_lm_l12_v2"
        self.url = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1/embeddings"
        self.headers = {
          'Authorization': f"Bearer {API_TOKEN}",
          'Content-Type': 'application/json',
        }
        
    def __call__(self, input_data: Documents) -> Embeddings:
        payload = {
            "input": input_data,
            "model": self.model_name,
        }

        req = requests.post(url=self.url, json=payload, headers=self.headers)
        res = req.json()
        data = res["data"]
        embeddings = [np.array(x["embedding"]) for x in data]
        
        return embeddings

In [9]:
multilingual_gemma2_collection = client.get_or_create_collection(
    name="multilingual-gemma2",
    embedding_function=MultinligualGemma2()
)

print(multilingual_gemma2_collection)
print(multilingual_gemma2_collection._embedding_function)

Collection(name=multilingual-gemma2)


In [10]:
df.head()

,title,url,type,url_text
0,informaticien cfc suisse,www.orientation.ch/dyn/show/1900?id=152,apprentissage,Informaticien CFC / Informaticienne CFC - orie...
1,école techniques des métiers de Lausanne,https://www.etml.ch/formation/em/informaticien...,école professionelle,Apprentissage d'informaticien·ne - ETML Nous u...
2,école des métiers Fribourg,https://emf.ch/formation/informaticien-ne-cfc,école professionelle,Informaticien-ne CFC · Ecole des Métiers Fribo...
3,centre professionel du Nord Vaudois,https://www.cpnv.ch/formations/ecole-metiers/i...,école professionelle,Informaticien-ne CFC - CPNV Aller au contenu M...
4,université de fribourg,www.unifr.ch/inf/fr/informatique,université,Informatique | Département d'informatique | U...


In [11]:
multilingual_gemma2_collection.add(
    ids=ids,
    metadatas=metadatas,
    documents=documents,
)

In [12]:
multilingual_gemma2_collection.count()

30

In [13]:
results = multilingual_gemma2_collection.query(
    query_texts=["j'aimerais faire l'université"],
    n_results=5,
)   

results["ids"]

[['42 lausanne',
  'informaticen cfc',
  'informaticien cfc suisse',
  'informaticen es',
  'école polytechnique fédérale de lausanne ']]

## BM25 

In [14]:
bm_25_collection = client.get_or_create_collection(
    name="bm25",
    embedding_function=Bm25EmbeddingFunction()
)

print(bm_25_collection)
print(bm_25_collection._embedding_function)

/home/matt/Projects/py/hackhaton_lab/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Collection(name=bm25)


In [15]:
ids: list[str] = df["title"].to_list() #df.index.astype("str").to_list()
documents: list[str] = df["url_text"].to_list()

# NOT WOKRING: WHY?
bm_25_collection.add(
    ids=ids,
    documents=documents,
)

TypeError: argument 'embeddings': 'dict' object cannot be converted to 'Sequence'

## Sentence Transformer

For all the HugginFace model.

In [16]:
# model we are interested in 
mini_lm_l12 = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" # 118M
mpnet_base = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # 278M
distule_base = "sentence-transformers/distiluse-base-multilingual-cased-v2" # 135M

In [17]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(mini_lm_l12)
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [18]:
%%time

sentences = ["yet another sequence to test the model", "This is a long sentence to demo how fast the model is and see if we can run it on CPU or maybe we need GPU?"]
embeddings = model.encode(sentences)

CPU times: user 128 ms, sys: 8.89 ms, total: 136 ms
Wall time: 38.1 ms


In [19]:
class SentenceTransformerFunction(EmbeddingFunction):
    def __init__(self, model_name: str) -> None:
        self.model_name = model_name
        self.model = SentenceTransformer(self.model_name)
        
    def __call__(self, input_data: Documents) -> Embeddings:
        embeddings = self.model.encode(input_data)
        return embeddings

In [20]:
mini_lm_l12_collection = client.get_or_create_collection(
    name="mini_lm_l12",
    embedding_function=SentenceTransformerFunction(mini_lm_l12),
)

print(mini_lm_l12_collection)
print(mini_lm_l12_collection._embedding_function)

Collection(name=mini_lm_l12)


In [21]:
%%time

mini_lm_l12_collection.add(
    ids=ids,
    metadatas=metadatas,
    documents=documents,
)

CPU times: user 2.69 s, sys: 72.7 ms, total: 2.77 s
Wall time: 693 ms


In [22]:
results = mini_lm_l12_collection.query(
    query_texts=["j'aimerais faire un apprentissage cfc"],
    n_results=10,
)   

results["ids"]

[['informaticien cfc suisse',
  "haute école d'ingénierie et de gestion du canton de vaud informatique logiciel",
  'haute école Arc informatique et systèmes de communication',
  'centre professionel du Nord Vaudois',
  "haute école d'ingénierie et de gestion du canton de vaud réseaux et systèmes",
  'informatique université de genève',
  'groupe mutuel',
  '42 lausanne',
  'swisscom',
  'haute école spécialisée de suisse occidentale sion']]

In [23]:
results = mini_lm_l12_collection.query(
    query_texts=["j'aimerais faire des hautes études"],
    n_results=10,
)   

results["ids"]

[['informatique université de genève',
  'haute école Arc informatique et systèmes de communication',
  'informatique de gestion université de fribourg',
  'université de fribourg',
  'groupe mutuel',
  'haute école spécialisée de suisse occidentale sion',
  'informaticien cfc suisse',
  "haute école d'ingénierie et de gestion du canton de vaud réseaux et systèmes",
  "haute école d'ingénierie et de gestion du canton de vaud informatique logiciel",
  '42 lausanne']]

In [24]:
results = mini_lm_l12_collection.query(
    query_texts=["les études universitaires m'intéresse le plus"],
    n_results=10,
)   

results["ids"]

[['informatique université de genève',
  'université de fribourg',
  'informatique de gestion université de fribourg',
  'haute école Arc informatique et systèmes de communication',
  'groupe mutuel',
  'informaticien cfc suisse',
  '42 lausanne',
  'haute école spécialisée de suisse occidentale sion',
  "haute école d'ingénierie et d'architecture fribourg",
  "haute école d'ingénierie et de gestion du canton de vaud réseaux et systèmes"]]